In [63]:
import numpy as np
import pickle
from typing import *
import os 
import tiktoken # for token counting
import json
from collections import defaultdict

In [64]:
def convert_to_prompt_tokens(list: List[int]) -> List[int]:
    '''
    multiply by 100 and remove decimal places
    '''

    return [int(x * 100) for x in list]

def _construct_demonstations(path) -> List[dict]:
    '''
    Construct the demonstrations for the LLM
    '''
    demonstation_prompt: List[dict] = []

    demonstation_folder = path

    for file in os.listdir(demonstation_folder):
        # open each folder and load in the demonstrations
        if os.path.isdir(demonstation_folder + '/' + file):
            for demo in os.listdir(demonstation_folder + '/' + file):
                if demo.endswith('.pkl'):
                    with open(demonstation_folder + '/' + file + '/' + demo, 'rb') as f:
                        data = pickle.load(f)

                        goal_tokens = convert_to_prompt_tokens(data['config']['goal'])
                        initial_positions_tokens = convert_to_prompt_tokens(data['config']['initial_position'])

                        ee_states: List[List[int]] = data['data']['ee_states']
                        ee_states_tokens = []
                        for i in range(len(ee_states)):
                            ee_states_tokens.append(convert_to_prompt_tokens(ee_states[i]))


                        demonstation_prompt.append({
                            'goal': goal_tokens,
                            'initial_position': initial_positions_tokens,
                            'ee_states': ee_states_tokens
                        })

    
    return demonstation_prompt

In [65]:
data = _construct_demonstations('demonstration_dataset_1500')


In [66]:
SYSTEM_ROLE = "You are a waypoint generator from the starting coordinates (first 2 values) to the goal (last 2 values). The robot has two arms of lengths 1 and 1 and operates on an X,Y plane."
JSONL_FILE_PATH = 'demonstration_runs.jsonl'

In [67]:
conversion_list = []

for i in range(len(data)):
    starting_position = data[i]['initial_position']
    goal = data[i]['goal']
    starting_obs = str(starting_position) + ' -> ' + str(goal)
    action_token_trajectory = data[i]['ee_states']

    conversation = {"messages": [{"role": "system", "content": str(SYSTEM_ROLE)}, {"role": "user", "content": str(starting_obs)}, {"role": "assistant", "content": str(action_token_trajectory)}]}
    conversion_list.append(conversation)

print(conversion_list[0])

# Define the file path for the JSONL file
with open(JSONL_FILE_PATH, 'w') as file:
    for conversation in conversion_list:
        json.dump(conversation, file)
        file.write('\n')  # Add a newline at the end of each JSON object

{'messages': [{'role': 'system', 'content': 'You are a waypoint generator from the starting coordinates (first 2 values) to the goal (last 2 values). The robot has two arms of lengths 1 and 1 and operates on an X,Y plane.'}, {'role': 'user', 'content': '[79, 40] -> [58, 76]'}, {'role': 'assistant', 'content': '[[83, 48], [78, 56], [72, 64], [65, 71]]'}]}


# Validation

In [68]:
# Load the dataset
with open(JSONL_FILE_PATH, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 1500
First example:
{'role': 'system', 'content': 'You are a waypoint generator from the starting coordinates (first 2 values) to the goal (last 2 values). The robot has two arms of lengths 1 and 1 and operates on an X,Y plane.'}
{'role': 'user', 'content': '[79, 40] -> [58, 76]'}
{'role': 'assistant', 'content': '[[83, 48], [78, 56], [72, 64], [65, 71]]'}


In [69]:
# Format error checks
format_errors = defaultdict(int)

for i,ex in enumerate(dataset):
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
            # print("error in message", i, message)
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
            # print("error in message", i, message)
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [70]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [71]:
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")


Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 78, 259
mean / median: 172.08733333333333, 169.0
p5 / p95: 108.0, 241.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 6, 187
mean / median: 100.08733333333333, 97.0
p5 / p95: 36.0, 169.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


In [72]:
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~258131 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~774393 tokens
